In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [2]:
URL = 'https://www.numbeo.com/cost-of-living/country_result.jsp?country=Netherlands'
response = get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
city_form = soup.find('form', {'class': 'standard_margin'})
city_form = [values['value'] for values in city_form('option')]
meu_01=[]

In [3]:
for city in city_form[:]:
    URL = 'https://www.numbeo.com/cost-of-living/city-estimator/in/'
    IRL = '?Recalculated=Submit+to+Recalculate&displayCurrency=EUR&members=1&restaurants_percentage=10.0&inexpensive_restaurants_percentage=50.0&drinking_coffee_outside=100.0&going_out_monthly=4.2&smoking_packs_per_day=0.0&alcoholic_drinks=25.0&type_of_food=0&driving_car=0.0&taxi_consumption=0.0&paying_for_public_transport=Monthly%2C+All+Members&sport_memberships=100.0&vacation=0.0&clothing_and_shoes=50.0&rent=none&kindergarten_count=0&private_schools_count=0'
    benefit=[]
    price=[]
    city = city.replace(' ','-')
    city = city.replace('(','')
    city = city.replace(')','')
    try:
        link = URL+city+IRL
        response = get(URL+city+IRL)
        soup = BeautifulSoup(response.text, 'html.parser')
        if len(soup.find_all('nav',{'class':'breadcrumb'}))==0:
            link=URL+city+'-Netherlands'+IRL
            response = get(link)
            soup = BeautifulSoup(response.text, 'html.parser')
        else:
            pass
        benefit = (soup.find_all('th',{'class':'tr_highlighted_menu'})[-1].get_text())
        price = (soup.find_all('th',{'class':'th_no_highlight_a_right'})[-1].get_text())
        meu_01.append({'benefit':benefit,'price':price,'city':city,'url':link})
    except:
        pass

In [4]:
df = pd.DataFrame(meu_01)
df = df[df['price']!=' ?'].reset_index(drop=True)
df['price'] = df['price'].str.replace('?','0',regex=True)
df['price'] = df['price'].str.replace('\D','',regex=True)
df['price'] = (df['price'].astype(float))/100
df.loc[df['city']=='The-Hague-Den-Haag','city'] = 'Den Haag'

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Your_Name")
df_list = df['city'].unique() + " Nederland"
address=[]
latitude=[]
longitude=[]
for i in range(len(df_list)):
    location = geolocator.geocode(df_list[i])
    address.append(location.address)
    latitude.append(location.latitude)
    longitude.append(location.longitude)

In [25]:
df['address'] = address
df['latitude'] = latitude
df['longitude'] = longitude

df['latitude'] = round(df['latitude'],7)
df['longitude'] = round(df['longitude'],7)

In [26]:
df['benefit'] = 'costs'
df['link'] = "<a target='_blank' href=" + (df['url']).astype(str) + ">" + 'Source' + "</a>"
df = df.drop_duplicates().sort_values('price', ascending=False).reset_index(drop=True)

In [27]:
import numpy as np
def haversine(lat1, lon1, lat2, lon2):
    earth_radius=6371
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    a = np.sin((lat2-lat1)/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin((lon2-lon1)/2.0)**2
    return earth_radius * 2 * np.arcsin(np.sqrt(a))

df['latitude1'] = 51.907183
df['longitude1'] = 4.472815
df['distance'] = haversine(df['latitude'], df['longitude'], df['latitude1'],df['longitude1'])
df['distance'] = round(df['distance'],2)

In [28]:
df

,benefit,price,city,url,address,latitude,longitude,link,latitude1,longitude1,distance
0,costs,1007.39,Alkmaar,https://www.numbeo.com/cost-of-living/city-est...,"'s-Hertogenbosch, Noord-Brabant, Nederland, 52...",51.688939,5.303116,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,62.04
1,costs,966.99,Amersfoort,https://www.numbeo.com/cost-of-living/city-est...,"Alkmaar, Noord-Holland, Nederland",52.600854,4.817099,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,80.61
2,costs,965.99,Zandvoort,https://www.numbeo.com/cost-of-living/city-est...,"Almere, Flevoland, Nederland",52.365710,5.221043,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,72.16
3,costs,941.47,Voorburg,https://www.numbeo.com/cost-of-living/city-est...,"Amersfoort, Utrecht, Nederland",52.163774,5.396588,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,69.33
4,costs,933.80,Roermond,https://www.numbeo.com/cost-of-living/city-est...,"Amstelveen, Noord-Holland, Nederland",52.302558,4.846787,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,50.84
5,costs,926.73,Amsterdam,https://www.numbeo.com/cost-of-living/city-est...,"Amsterdam, Noord-Holland, Nederland",52.372760,4.893604,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,59.20
6,costs,916.87,Haarlem,https://www.numbeo.com/cost-of-living/city-est...,"Best, Noord-Brabant, Nederland",51.510017,5.398662,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,77.59
7,costs,913.83,Amstelveen,https://www.numbeo.com/cost-of-living/city-est...,"Breda, Noord-Brabant, Nederland",51.588785,4.776024,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,41.10
8,costs,912.78,Ede,https://www.numbeo.com/cost-of-living/city-est...,"Delft, Zuid-Holland, Nederland",51.999457,4.362724,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,12.74
9,costs,912.40,Utrecht,https://www.numbeo.com/cost-of-living/city-est...,"Deventer, Overijssel, Nederland",52.269574,6.236340,<a target='_blank' href=https://www.numbeo.com...,51.907183,4.472815,127.05


In [9]:
ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds
ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds
ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds

NameError: name 'ksjdhakjsdhkjashdkjashdkjahskdjhaskjdhaskjds' is not defined

In [10]:
df.reset_index(drop=True).to_csv('../../app/costs.csv')

In [ ]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd

In [ ]:
def  minha(URL,city):
    response = get(URL)
    soup = BeautifulSoup(response.text, 'html.parser')
    split1 = soup.find_all('tr',{'class':'tr_standard'})
    benefit = []
    col_factor = []
    price = []
    city_form=[]
    for i in range(len(split1)):
        try:
            benefit_ = split1[i].find('td').find(text=True)
        except:
            benefit_ = None
        try:
            col_factor_ = split1[i].find('span',{'class':'estimator_multiplier'}).find(text=True)
        except:
            col_factor_ = None
        try:
            price_ = split1[i].find('span',{'class':'price_in_units_for_analysis'}).find(text=True)
        except:
            price_ = None

        benefit.append(benefit_)
        col_factor.append(col_factor_)
        price.append(price_)
        city_form.append(city)
        
    return [benefit,col_factor,city_form,price]

In [ ]:
URL = 'https://www.numbeo.com/cost-of-living/country_result.jsp?country=Netherlands'
response = get(URL)
soup = BeautifulSoup(response.text, 'html.parser')
city_form = soup.find('form', {'class': 'standard_margin'})
city_form = [values['value'] for values in city_form('option')]
meu_01=[]

In [ ]:
for city in city_form[:]:
    URL = 'https://www.numbeo.com/cost-of-living/city-estimator/in/'+city
    my_dict_ = minha(URL,city)
    meu_01.append({'benefit':my_dict_[0],'col_factor':my_dict_[1],'city':my_dict_[2],'price':my_dict_[3]})

In [ ]:
df = pd.DataFrame({})
for i in range(len(meu_01)):
    df = pd.concat([df, pd.DataFrame(meu_01[i])],0)

In [ ]:
df['price'] = df['price'].str.replace('?','0',regex=True)
df['price']= df['price'].str.replace('\D','',regex=True)
df['price'] = (df['price'].astype(float))/100

In [ ]:
df['col_factor']= df['col_factor'].str.replace('x','',regex=True)
df['col_factor'] = (df['col_factor'].astype(float))/2

---

In [ ]:
# Import Libs
from folium.plugins import FastMarkerCluster
import io
import base64
import folium
import pandas as pd
import streamlit as st
from st_aggrid import AgGrid
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import folium.plugins
import branca
import branca.colormap as cm
import folium.plugins
import branca
import branca.colormap as cm

In [ ]:
final = pd.read_csv('../../app/costs.csv', index_col=[0])
final_ = final[['price', 'city', 'url']].to_html(escape=False)
lats = final['latitude'].tolist()
lons = final['longitude'].tolist()
city = final['city'].tolist()
price = final['price'].tolist()
map = folium.Map(location=[52.2129919, 5.2793703],zoom_start=7, tiles="cartodbdark_matter")
feature_group = folium.FeatureGroup("Locations")
for lats, lons, price, city in zip(lats, lons, price, city):
    feature_group.add_child(
        folium.CircleMarker(
            location=[lats, lons],
            popup=f"{city}, €{price}",
            radius=4,
            fill=True,
        )
    )
map.add_child(feature_group)

In [ ]:
fname = 'test_colorbar.html'


base_lat = 50.682294344
base_lon = 10.939628989
f_map = folium.Map(location=[base_lat,base_lon], tiles="OpenStreetMap", zoom_start=17)

lat_ = [50.6823, 50.6822, 50.6821, 50.6820, 50.6819]
lon_ = [10.9396291, 10.9396279, 10.9396269, 10.9396265, 10.9396261]
acc_ = [11.0,44.5,149.9,319.1,540.0,752.6]

colormap = cm.LinearColormap(colors=['darkblue', 'blue', 'cyan', 'yellow', 'orange', 'red'],
                             index=[0, 25, 62.5, 156.25, 390.6, 1000], vmin=0, vmax=1000,
                             caption='Total Standard deviation at the point[mm]')

fg = folium.FeatureGroup(name=fname.split('.')[0])    

for pt in range(len(lat_)):

    color = colormap(acc_[pt])
    fg.add_child(folium.CircleMarker(location=[lat_[pt],lon_[pt]],
                                     radius=6,
                                     fill=True,
                                     color=color,
                                     fill_color=color))

f_map.add_child(fg)
f_map.add_child(colormap)
f_map

In [ ]:
final = pd.read_csv('../../app/costs.csv', index_col=[0])

map = folium.Map(location=[52.2129919, 5.2793703],zoom_start=7, tiles="cartodbdark_matter")
feature_group = folium.FeatureGroup("Locations")

lats = final['latitude'].tolist()
lons = final['longitude'].tolist()
city = final['city'].tolist()
price = final['price'].tolist()
acc_ = final['price'].unique().tolist()

for lats, lons, price, city in zip(lats, lons, price, city):
    color = colormap(acc_.index(price))
    feature_group.add_child(
        folium.CircleMarker(
            location=[lats, lons],
            radius=6,
            fill=True,
            color=color,
            fill_color=color))
map.add_child(feature_group)

In [ ]:
for lat, lon, traffic_q, traffic, bike, city in zip(df['latitude'], df['longitude'], df['traffic_index_quartile'], df['traffic_index'], df['bike_score'], df['city']):
    folium.CircleMarker(
        [lat, lon],
        radius=.15*bike,
        popup = ('City: ' + str(city).capitalize() + '<br>'
                 'Bike score: ' + str(bike) + '<br>'
                 'Traffic level: ' + str(traffic) +'%'
                ),
        color='b',
        key_on = traffic_q,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[traffic_q],
        fill=True,
        fill_opacity=0.7
        ).add_to(traffic_map)
traffic_map

In [ ]:
import seaborn as sns
print(sns.palplot(sns.color_palette("coolwarm", 33)))
print(sns.palplot(sns.color_palette("flare", 33)))
print(sns.palplot(sns.color_palette("viridis", 33)))
print(sns.palplot(sns.color_palette("Spectral", 33)))




In [ ]:
up = sns.palplot(sns.color_palette("Spectral", 33))

In [ ]:
rgb = [(0.15, 0.25, 0.35), (0.35, 0.55, 0.51), (0.5, 0.95, 0.34)]
= sns.palplot(sns.blend_palette(rgb, n_colors=10))


In [ ]:
sns.deep.values

In [ ]:
pal = sns.color_palette("magma")
print(pal)
sns.palplot(pal)

In [ ]:
color_pallete = (sns.color_palette("Spectral", 33))
color_pallete = color_pallete.as_hex()
c = color_pallete[1]
c